In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy import integrate
from scipy.optimize import fmin
import pandas as pd
from math import hypot
from scipy.optimize import curve_fit
import matplotlib.image as mpimg

In [ ]:
# for irregular cross section:
# import csv of surveys
fname1 = "xs1.csv" # define file name, must be in working directory
xs1 = pd.read_csv(fname1) # read into pandas dataframe
x = xs1['Station (m)'] # define x as station, depends on actual collumn name
y = xs1['Elev (m)'] # define y as elevation, depends on actual collumn name (execute xs1.columns for dataframe column names)

# read second cross section, iterate for multiple cross sections
##fname2 = "xs2.csv"
##xs2 = pd.read_csv(fname)

# calculate cross sectional area for several WSE
f2 = interp1d(x, y, kind='cubic') # use cubic interpolation to define continuous cross section function

xnew = np.linspace(min(x), max(x), num=1000, endpoint=True) # create index array for x
ynew = np.linspace(min(y), max(y), num=100, endpoint=True) # create index array for y
area = []
depth = []
p = []

# define function to calculate wetted perimeter
## https://stackoverflow.com/questions/46098157/how-to-calculate-the-length-of-a-curve-of-a-math-function
def arclength(f, a, b, tol=1e-6):
    nsteps = 1  # number of steps to compute
    oldlength = 1.0e20
    length = 1.0e10
    while abs(oldlength - length) >= tol:
        nsteps *= 2
        fx1 = f(a)
        xdel = (b - a) / nsteps  # space between x-values
        oldlength = length
        length = 0
        for i in range(1, nsteps + 1):
            fx0 = fx1  # previous function value
            fx1 = f(a + i * (b - a) / nsteps)  # new function value
            length += hypot(xdel, fx1 - fx0)  # length of small line segment
    return length

for i in ynew:
    y_0=[i for j in range(1000)] # create water surface array
    idx = np.argwhere(np.diff(np.sign(f2(xnew) - y_0))).flatten() # find intersection between water surface and cross section
    auc = integrate.quad(f2, (xnew[min(idx)]), (xnew[max(idx)])) # find area under xs curve
    rec = (max(xnew[idx])-min(xnew[idx]))*i # find area in rectangle 
    xsa = rec-auc[0] # subtract auc from rectangle to find area in cross section
    area.append(xsa)
    p.append(arclength(f2, (xnew[min(idx)]), (xnew[max(idx)]), 1e-3))

# find depth to lowest point or thalweg
#t = fmin(f2,1) # returns incorrect value, why??
t = min(y) # define thalweg elevation
t = [t for i in ynew]    
depth = ynew-t 

R = [i / j for i, j in zip(area, p)]

fig, ax1 = plt.subplots()

ax2 = ax1.twinx()
ax1.plot(area, depth,'r')
ax2.plot(area, R,'b')

ax1.set_xlabel('station(m)')
ax1.set_ylabel('depth(m)',color='r')
ax2.set_ylabel('hydraulic radius (m)',color='b')

plt.show()

# create power relationship between R and h
def func(x, m, c):
    return c*x**m

xdata = depth[depth!=0]
ydata = area[area!=0]

popt, pcov = curve_fit(func, xdata, ydata)
plt.plot(xdata, func(xdata, *popt), 'r-')
#plt.plot(depth,area,'b-')
plt.show()

plt.plot(depth,R)
plt.show

In [ ]:
# create power relationship between R and h

def func(x, m, c):
    return c*x**m

xdata = depth[depth!=0]
ydata = area[area!=0]

popt, pcov = curve_fit(func, xdata, ydata)
plt.plot(xdata, func(xdata, *popt), 'r-')

plt.show()

In [ ]:
# example power fit, cite then DELETE

def func(x, a, b, c):
    return a * np.exp(-b * x) + c

xdata = np.linspace(0, 4, 50)
y = func(xdata, 2.5, 1.3, 0.5)
rng = np.random.default_rng()
y_noise = 0.2 * rng.normal(size=xdata.size)
ydata = y + y_noise
plt.plot(xdata, ydata, 'b-', label='data')

popt, pcov = curve_fit(func, xdata, ydata)

plt.plot(xdata, func(xdata, *popt), 'r-',
         label='fit: a=%5.3f, b=%5.3f, c=%5.3f' % tuple(popt))

popt, pcov = curve_fit(func, xdata, ydata, bounds=(0, [3., 1., 0.5]))

plt.plot(xdata, func(xdata, *popt), 'g--',
         label='fit: a=%5.3f, b=%5.3f, c=%5.3f' % tuple(popt))

plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()